In [1]:
import utility as ut

OPENAI_API_KEY = ut.OPENAI_API_KEY
LANGSMITH_API_KEY = ut.LANGSMITH_API_KEY
MODEL_NAME = ut.MODEL_NAME

TOOLS PROVIDE THE "ACTION" PART of ReACT
THEIR RESULTS ARE OBSERVATIONS

TOOLS ALLOW AGENTS TO ACT IN THE REAL WORLD. CAREFUL DESCRIPTIONS CAN HELP YOUR AGENT DISCOVER HOW TO USE THESE TOOLS 


# CALCULATOR EXAMPLE 
IN THIS EXAMPLE, THE DOCSTRING AND INFERRED ARGUEMENTS AND ARGUEMENT TYPES ARE USED BY LLM TO DETERMINE WHEN AND HOW TO CALL THE TOOL 

In [2]:
from typing import Literal

from langchain.tools import tool

@tool 
def real_number_calculator(
    a: float, 
    b: float,
    operation: Literal["add","subtract","multiply","divide"]
) -> float:
    """ Perform basic arithematic operations on two real numbers. """

    print("Invoking calculator tool")
    if operation == "add":
        return a+b
    elif operation == "subtract":
        return a-b
    elif operation == "multiply":
        return a*b
    elif operation == "divide":
        if b == 0:
            raise ValueError("Division by zero not allowed")
        return a/b 
    else:
        raise ValueError(f"Invalid Operation:{operation}")

In [3]:
from langchain.agents import create_agent
agent = create_agent(
    model = MODEL_NAME,
    tools = [real_number_calculator],
    system_prompt = "You are an helpful assistant",
)

In [5]:
result = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "what is 3.1125 * 4.1234?"
            }
        ]
    }
)
print(result["messages"][-1].content)

Invoking calculator tool
The result of \( 3.1125 \times 4.1234 \) is \( 12.8340825 \).


In [6]:
result = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "what is 3.1125 * 4.1234 + 3.1333?"
            }
        ]
    }
)
print(result["messages"][-1].content)

Invoking calculator toolInvoking calculator tool

Invoking calculator tool
The result of \( 3.1125 \times 4.1234 + 3.1333 \) is approximately \( 15.9674 \).


In [7]:
result = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "what is 3*4?"
            }
        ]
    }
)
print(result["messages"][-1].content)

Invoking calculator tool
The result of \( 3 \times 4 \) is \( 12 \).


# ADDING A MORE DETAILED DESCRIPTION 
While a basic description if often sufficient, Langchain has support for enhanced descriptions. 
The example below uses one method: Google Style arguement descriptions. 
used with `parse_docstring=True`, this will parse and pass the arg descriptions to the model. you can rename the tool and change its description. 
This can be effective when you are sharing a standard tool but would like agent-specific instructions

In [9]:
from typing import Literal 
from langchain.tools import tool 

@tool(
    "calculator",
    parse_docstring=True,
    description = (
        "Perform basic arthematic operations on two real numbers."
        "Use this whenever you have operations on any numbers, even if they are integers."
    )
)
def real_number_calculator(
    a: float, 
    b: float,
    operation: Literal["add", "subtract", "multiply", "divide"]
) -> float:
    """ Perform basic arithematic operations on two real numbers. 
    
    Args:
        a (float): The first number
        b (float): The second number
        operation (Literal["add", "subtract", "multiply", "divide"]):
            The arithematic operations to perform 

            - `"add"`: Returns the sum of `a` and `b`
            - `"subtract"`: Returns the result of `a - b`
            - `"multiply"`: Returns the result of `a * b`
            - `"divide"`: Returns the result of `a / b`. Raises error if `b` is zero

    Returns:
        float: the numerical result of the specified operation

    Raises:
        ValueError: If an invalid operation is provided or division by zero is attempted
    """

    if operation == "add":
        return a+b
    elif operation == "subtract":
        return a-b
    elif operation == "multiply":
        return a*b
    elif operation == "divide":
        if b == 0:
            raise ValueError("Division by zero not allowed")
        return a/b 
    else:
        raise ValueError(f"Invalid Operation:{operation}")


In [10]:
from langchain.agents import create_agent
agent = create_agent(
    model = MODEL_NAME,
    tools = [real_number_calculator],
    system_prompt = "You are an helpful assistant",
)

In [11]:
result = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "what is 3.1125 * 4.1234 + 3.1333?"
            }
        ]
    }
)
print(result["messages"][-1].content)

The result of \( 3.1125 \times 4.1234 + 3.1333 \) is approximately \( 15.9674 \).
